In [7]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

from dlsvm import utils, layers

import torch
from torch import Tensor as T
import torch.nn as nn
from torch import autograd
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm_notebook


import math
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
torch.cuda.set_device(0)

### MNIST

In [9]:
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

batch_size = 128
n_epochs = 1

In [10]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, 3)
        self.conv2 = nn.Conv2d(64, 128, 3)
        self.pool1 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(128 * 12 * 12, 512)
        self.kernel = layers.KernelLayer(512, 1536)
        self.fc2 = nn.Linear(1536, 10)

        # print('Leaky ReLU is used!')
        
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool1(x)

        x = x.view(x.size(0), -1)

        x = self.fc1(x)
        x = F.relu(x)

        x = self.fc2(x)
        x = F.softmax(x)
        return x

In [11]:
train_loader = DataLoader(
    datasets.MNIST('../../RandomNet/data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True)
test_loader = DataLoader(
    datasets.MNIST('../../RandomNet/data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=False)

In [12]:
class SVMNet(Net):
    def __init__(self):
        super(SVMNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, 3, bias=False)
        self.conv2 = nn.Conv2d(64, 128, 3, bias=False)
        
        self.pool1 = nn.MaxPool2d(2)
        
        self.fc1 = nn.Linear(128 * 12 * 12, 512)        
        self.fc2 = nn.Linear(512, 10)

In [13]:
def train(train_loader, test_loader, model, optimizer, scheduler=None, start_epoch=0, stop_epoch=20,
          fix_random_weights=False):

    n_epochs = stop_epoch - start_epoch
    n_steps = 0
    for epoch in tqdm_notebook(range(start_epoch, stop_epoch), desc='epochs', total=n_epochs):

        # train
        if fix_random_weights:
            model.eval()
        else:
            model.train()
        train_loss = 0
        n_train_samples = 0
        n_train_batches = 0
        if scheduler is not None:
            scheduler.step()
        for data, target in tqdm_notebook(train_loader, leave=False):
            data, target = data.cuda(), target
            target_onehot = torch.zeros(target.size()[0], 10)
            target_onehot = target_onehot.scatter_(1, target.view(-1, 1), 1).cuda()
            data, target = autograd.Variable(data), autograd.Variable(target_onehot)

            optimizer.zero_grad()
            output = model(data)
            loss = utils.svm_l1loss(output, target, model.fc2.weight, C=1, batch_fraction=1)
            loss.backward()
            optimizer.step()

            train_loss += loss.data[0]
            n_train_samples += target.size(0)
            n_train_batches += 1
            n_steps += 1

        train_loss /= n_train_batches


        # evaluate
        test_accuracy = utils.evaluate(model, test_loader)

        # print progress
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tTest acc: {:.3f}'.format(
              epoch, n_train_samples, len(train_loader.dataset),
              100. * n_train_batches / len(train_loader), train_loss,
              test_accuracy))

In [17]:
model = SVMNet()
model.cuda()

optimizer = optim.Adam(model.parameters(), lr=0.001)
train(train_loader, test_loader, model, optimizer, start_epoch=0, stop_epoch=20)

[autoreload of dlsvm.layers failed: Traceback (most recent call last):
  File "/home/yeahrmek/miniconda3/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 246, in check
    superreload(m, reload, self.old_objects)
  File "/home/yeahrmek/miniconda3/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 385, in superreload
    update_generic(old_obj, new_obj)
  File "/home/yeahrmek/miniconda3/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 324, in update_generic
    update(a, b)
  File "/home/yeahrmek/miniconda3/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 289, in update_class
    if update_generic(old_obj, new_obj): continue
  File "/home/yeahrmek/miniconda3/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 324, in update_generic
    update(a, b)
  File "/home/yeahrmek/miniconda3/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 267, in update_function
    setattr(old, name, get

Train Epoch: 0 [60000/60000 (100%)]	Loss: 1208.670956	Test acc: 0.675


Train Epoch: 1 [60000/60000 (100%)]	Loss: 1186.456445	Test acc: 0.783


Train Epoch: 2 [60000/60000 (100%)]	Loss: 1174.197747	Test acc: 0.876


Train Epoch: 3 [60000/60000 (100%)]	Loss: 1167.465328	Test acc: 0.883


...

Train Epoch: 15 [60000/60000 (100%)]	Loss: 1152.413048	Test acc: 0.988


Train Epoch: 16 [60000/60000 (100%)]	Loss: 1152.262680	Test acc: 0.990


Train Epoch: 17 [60000/60000 (100%)]	Loss: 1152.326875	Test acc: 0.989


Train Epoch: 18 [60000/60000 (100%)]	Loss: 1152.347993	Test acc: 0.990


Train Epoch: 19 [60000/60000 (100%)]	Loss: 1152.218038	Test acc: 0.991



In [90]:
for name, p in model.named_parameters():
    if 'mu' in name or 'std' in name:
        if 'std' in name:
            x = np.exp(p.data.cpu().numpy())
        else:
            x = p.data.cpu().numpy()
        print(name, x[0])

conv1.W_mu 0.0764163
conv1.W_logstd 0.0099965
conv2.W_mu 0.051796
conv2.W_logstd 0.0100005


In [66]:
train(train_loader, test_loader, model, optimizer, n_epochs=20, random_weights=True)

126/|/ 27%|| 126/469 [00:20<00:07, 43.81it/s]Train Epoch: 0 [60032/60000 (100%)]	Loss: 2.302593	Test acc: 0.065


Train Epoch: 1 [60032/60000 (100%)]	Loss: 2.302588	Test acc: 0.079


Train Epoch: 2 [60032/60000 (100%)]	Loss: 2.302584	Test acc: 0.105


Train Epoch: 3 [60032/60000 (100%)]	Loss: 2.302583	Test acc: 0.090


KeyboardInterrupt: 

178/|/ 38%|| 178/469 [00:16<00:06, 45.63it/s]